# Comparison of Martijns and Marius results of ion suppression correction


In [ ]:
import os
import pandas as pd
import numpy as np
import scanpy as sc
import seaborn as sns
from src.correction import *
from src import const
%matplotlib inline

Martijn implemented the correction in Rmd, his results are loaded from CSV files.

In [ ]:
def load_intensities_r(path, prefix = const.CELL_PRE):
    intensities_df = pd.read_csv(path)
    intensities_df.index = [ prefix + str(i) for i in intensities_df.cell]
    intensities_df.pop('cell')
    return(intensities_df)


raw_intensities_r = load_intensities_r('data/Martijns_results_C18H32O2/raw_intensities_r.csv').dropna()
cor_intensities_r = load_intensities_r('data/Martijns_results_C18H32O2/cor_intensities_r.csv').dropna()
cor_pixel_intensities_r = load_intensities_r('data/Martijns_results_C18H32O2/cor_pixel_intensities_r.csv', prefix=const.PIXEL_PRE).dropna()


I implemented the correction in Python. The resulting spatiomolecular matrices are compared on multiple levels:

- corrected ion intensities of individual pixels (before pixel-cell deconvolution)
- uncorrected ion intensities of cells (no IS correction but pixel-cell deconvolution)
- corrected ion intensities of cells (both IS correction and pixel-cell deconvolution)

By that, the source of possible deviations can be narrowed down to one of the processing steps.

In [ ]:
adata = sc.read('data/Mx_Co_Cultured/dataset_threshold_0.1/cells_spatiomolecular_adata.h5ad')
adata_cor = sc.read('data/Mx_Co_Cultured/dataset_threshold_0.1/cells_spatiomolecular_adata_corrected.h5ad')
am_adata_cor = sc.read('data/Mx_Co_Cultured/dataset_threshold_0.1/am_spatiomolecular_adata_corrected.h5ad')


In [ ]:
# only molecules and cells that occur in both datasets are compared.
included_molecules = adata.var.index.intersection(raw_intensities_r.columns).intersection(
    ad.read(os.path.join('data/Mx_Co_Cultured', "gen_batch_sm_matrix.h5ad")).var_names)
#included_molecules = ['C16H30O2', "C25H44NO7P", "C45H78NO8P"]
included_cells = adata.obs.index.intersection(raw_intensities_r.index)#[[26, 75, 433]]
included_pixels = am_adata_cor.obs.index.intersection(cor_pixel_intensities_r.index)#[[26, 75, 433]]

As a measure of deviation, I calculate the difference for every metabolite intensity for every cell/pixel. If the processing in R and Python is the same, these differences should all be close to 0.

In [ ]:
from matplotlib.pyplot import axis

def calc_diff_df(df1, df2):
    logfc_df = (df1.loc[included_cells, included_molecules] - df2.loc[included_cells, included_molecules]) / df1.loc[included_cells, included_molecules]
    #logfc_df = np.log10(logfc_df)
    #logfc_df = logfc_df.replace([np.nan, np.Inf, -np.Inf], 0)
    return(logfc_df)

raw_fc_df = calc_diff_df(adata.to_df(), raw_intensities_r)
cor_fc_df = calc_diff_df(adata_cor.to_df(), cor_intensities_r)

pixel_fc_df = am_adata_cor.to_df().loc[included_pixels, included_molecules] - cor_pixel_intensities_r.loc[included_pixels, included_molecules].replace(np.Inf, 0)
#pixel_fc_df = np.log10(pixel_fc_df).replace([np.nan, np.Inf, -np.Inf], 0)

The deviations are shown visually in a heatmap. As quantitative measures, the summed (absolute differences summed across whole dataset) and average absolute deviations (sum of all absolute differences divided by number of datapoints) are given.

### Corrected intensities of pixels

The isolated process of ion intensity correction has added deviations to a selected set of metabolites, that partly coincides with the set that was corrected using a reference metabolite as they lack enough valid data points for quantile regression. Specifically, the python-generated data yielded, if any, only higher values than the intensities processed by the R script.

In [ ]:
import plotly.express as px
from matplotlib_venn import venn2

def calc_differences(df, mean = None):
    
    df.index.name = "cells"
    df.columns.name = "ions"
    
    # fig = px.imshow(df.transpose())
    # fig.show()
    fig, ax = plt.subplots(figsize=(14, 4))
    sns.heatmap(df.T)

    #sns.clustermap(df)
    print('summed absolute difference of log fold changes: %1.2f' % np.abs(df).values.sum())
    print('average absolute difference of individual log fold change: %1.4f' % (np.abs(df).values.sum() / df.shape[0] / df.shape[1] ))
    if mean is not None:
        print('relative absolute deviation: %1.5f' % (np.abs(df).values.mean() / mean ))



calc_differences(pixel_fc_df, mean = am_adata_cor.to_df().loc[included_pixels, included_molecules].mean().mean())

In contrast, when observing the pixel-cell devonvolution independantly, deviations naturally occur rather for selected cells than metabolites. With multiple calculations of overlap-matrices and deconvolution weights, calculation inaccuracies could have possibly been amplified for some cells more than others. This step has been re-engineered after an internal SpaceM method by both Martijn and me and should be processed using the original method in the long run.

In [ ]:
calc_differences(raw_fc_df, mean = adata.to_df().loc[included_cells, included_molecules].mean().mean())

When correction and pixel-cell deconvolution are taken together, it can be expected that molecule-specific and cell-specific deviations add up. This can be observed precisely with both the heatmap and the average absolute deviation per data point. As the discrepancies generated by the ion-suppression correction are generally smaller than the ones resulting from (Martijns or my implementation of) pixel-cell deconvolution, I would accept them as they are.

In [ ]:
calc_differences(cor_fc_df, mean = adata_cor.to_df().loc[included_cells, included_molecules].mean().mean())

Lastly, the relative differences in every data point are calculated and plotted separately for every datapoint

In [ ]:
pixel_rel = (am_adata_cor.to_df().loc[included_pixels, included_molecules] - cor_pixel_intensities_r.loc[included_pixels, included_molecules].replace(np.Inf, 0)) / am_adata_cor.to_df().loc[included_pixels, included_molecules]
pixel_rel.index = ["px_" + i for i in pixel_rel.index]
pixel_rel
print("median relative deviation: %1.2f Percent" % (100*np.abs(pixel_rel).stack().median()))
print("min relative deviation: %1.2f Percent"% (100*(pixel_rel).stack().min()))
print("max relative deviation: %1.2f Percent" % (100*(pixel_rel).stack().max()))

In [ ]:
np.random.seed(123)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.kdeplot(pixel_rel.loc[:, np.random.choice(pixel_rel.columns, 14)], ax=ax1)
sns.kdeplot(pixel_rel.T.loc[:, np.random.choice(pixel_rel.index, 14)], ax=ax2)
ax1.set(ylabel="density", xlabel=const.LABEL['RDens'], title="Colored by ion")
ax2.set(ylabel="", xlabel=const.LABEL['RDens'], title="Colored by pixel")
fig.tight_layout()

In [ ]:
deconv_rel = (adata_cor.to_df().loc[included_cells, included_molecules] - raw_intensities_r.loc[included_cells, included_molecules].replace(np.Inf, 0)) / adata.to_df().loc[included_cells, included_molecules]
deconv_rel.index = ["cell_" + i for i in deconv_rel.index]

print("median relative deviation: %1.1f Percent" % (100*np.abs(deconv_rel).stack().median()))
print("min relative deviation: %1.1f Percent"% (100*(deconv_rel).stack().min()))
print("max relative deviation: %1.1f Percent" % (100*(deconv_rel).stack().max()))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
sns.kdeplot(deconv_rel.loc[:, np.random.choice(deconv_rel.columns, 14)], ax=ax1)
sns.kdeplot(deconv_rel.T.loc[:, np.random.choice(deconv_rel.index, 14)], ax=ax2)
ax1.set(ylabel="density", xlabel=const.LABEL['RDens'], title="Colored by ion")
ax2.set(ylabel="", xlabel=const.LABEL['RDens'], title="Colored by cell")
fig.tight_layout()